In [100]:
%pip install matplotlib --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [101]:
import matplotlib.pyplot as plt
import random
import os
import shutil
from IPython.display import display, clear_output

In [102]:
%matplotlib tk
# %matplotlib inline

In [103]:
shutil.rmtree('criacao_base', ignore_errors=True)
os.makedirs('criacao_base', exist_ok=True)

In [104]:
tamanho = (500, 500)
ponto_inicial_final = (0, 0), tamanho
raio_size = 25
visual_s = 5
qtd_centroides = 10
paciencia = 10

In [105]:
def inserir_circulo(ax, x, y, raio_size, visual_s, color_centro, color_ext):
    valores_adicionados = []
    valor_baixo = (x, y - raio_size)
    valor_cima = (x, y + raio_size)
    valor_esquerda = (x - raio_size, y)
    valor_direita = (x + raio_size, y)
    posicoes = (valor_baixo, valor_cima, valor_esquerda, valor_direita)
    
    [valores_adicionados.append(ax.scatter(*posicao, color=color_ext, s=visual_s)) for posicao in posicoes]
    valores_adicionados.append(ax.scatter(x, y, color=color_centro, s=visual_s))

    valores_adicionados.append(plt.Circle((x, y), raio_size, color=color_ext, fill=False))
    ax.add_patch(valores_adicionados[-1])

    posicao_circulo = (valor_baixo, valor_cima, valor_esquerda, valor_direita)
    return valores_adicionados, posicao_circulo

def exibir_figura(figura):
    # plt.waitforbuttonpress()
    # plt.pause(0.001)
    figura.show()

def remover_valores_pintados(valores_pintados):
    for valor_adicionado in valores_pintados:
        valor_adicionado.remove()

In [106]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.scatter(*ponto_inicial_final[0], color='green', s=visual_s)
ax.scatter(*ponto_inicial_final[1], color='red', s=visual_s);
plt.savefig('criacao_base/0_pontos_iniciais.png', bbox_inches='tight', pad_inches=0.1, dpi=150);

In [107]:
centroids = []
posicoes_centroides = []

interacao_atual = 0
paciencia_atual = 0

def distancia_euclidiana(ponto1, ponto2):
    x1, y1 = ponto1
    x2, y2 = ponto2
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

def verificar_distancia(x1, y1, x2, y2, raio_size) -> bool:
    return distancia_euclidiana((x1, y1), (x2, y2)) < 2 * raio_size

while len(centroids) < qtd_centroides and paciencia_atual < paciencia:

    interacao_atual += 1
    paciencia_atual += 1
    
    x = random.uniform(raio_size, tamanho[0] - raio_size)
    y = random.uniform(raio_size, tamanho[1] - raio_size)

    if verificar_distancia(x, y, *ponto_inicial_final[0], raio_size):
        print(f'Ponto ({x}, {y}) está dentro da área de influência do ponto inicial.')
    elif verificar_distancia(x, y, *ponto_inicial_final[1], raio_size):
        print(f'Ponto ({x}, {y}) está dentro da área de influência do ponto final.')
    elif any(verificar_distancia(centroid[0], centroid[1], x, y, raio_size) for centroid in centroids):
        print(f'Ponto ({x}, {y}) está dentro da área de influência de um dos centroides já criados.')
        valores_pintados, _ = inserir_circulo(ax, x, y, raio_size, visual_s, 'red', 'orange')
        exibir_figura(fig)
        remover_valores_pintados(valores_pintados)
    else:
        _, posicao_circulo = inserir_circulo(ax, x, y, raio_size, visual_s, 'blue', 'purple')
        exibir_figura(fig)

        centroids.append((x, y))
        posicoes_centroides.append(posicao_circulo)
        paciencia_atual = 0

exibir_figura(fig)

Ponto (45.29643131025084, 383.8720568933495) está dentro da área de influência de um dos centroides já criados.
Ponto (74.19169815656468, 100.47869140411828) está dentro da área de influência de um dos centroides já criados.
Ponto (328.6455339892529, 214.36178134749954) está dentro da área de influência de um dos centroides já criados.
Ponto (166.56999551801198, 351.2610941348105) está dentro da área de influência de um dos centroides já criados.


In [108]:
def coeficientes_reta(ponto_inicial, ponto_final):
    """
    Calcula os coeficientes da equação da reta que passa pelos pontos iniciais e finais.
    a -> coeficiente angular (inclinação)
    b -> coeficiente linear (intercepto)
    c -> termo constante

    link do exemplo: https://www.todamateria.com.br/equacao-da-reta/
    link da video aula: https://www.youtube.com/watch?v=9dhtGUPgekw

    é possivel transformar a equação da reta na forma y = mx + b, onde m é o coeficiente angular e b é o coeficiente linear.
    
    p1 = (-1, 8)
    p2 = (-5, -1)

    exemplo: 
    a = 9
    b = -4
    c = 41
    """
    
    a = (ponto_inicial[1] - ponto_final[1])
    b = (ponto_final[0] - ponto_inicial[0])
    c = (ponto_final[1] * ponto_inicial[0]) - (ponto_final[0] * ponto_inicial[1])
    return a, b, c

A = (0, 0)
B = (1, 1)
a, b, c = coeficientes_reta(A, B)


print(f"Coeficientes da reta entre {A} e {B}: a = {a}, b = {b}, c = {c}")
print(f"Equação da reta: {a}x + {b}y + {c} = 0")
print(f"Forma reduzida: y = {-a/b}x + {-c/b}")

Coeficientes da reta entre (0, 0) e (1, 1): a = -1, b = 1, c = 0
Equação da reta: -1x + 1y + 0 = 0
Forma reduzida: y = 1.0x + 0.0


In [109]:
def distancia_ponto_reta(novo_ponto, a, b, c):
    """
    Calcula a distância do ponto (novo_ponto) à reta definida pelos coeficientes a, b e c.
    A fórmula é dada por: |ax + by + c| / sqrt(a^2 + b^2)

    Link exemplo: https://www.todamateria.com.br/distancia-entre-dois-pontos/
    Link video aula: https://www.youtube.com/watch?v=FSfwY1fM4EI
    
    Por que não usar a formula: 
    '
    distancia_x = x2 - x1
    distancia_y = y2 - y1
    distancia_euclidiana = (distancia_x ** 2 + distancia_y ** 2) ** 0.5
    '

    Motivo: 
    A fórmula da distância entre dois pontos é uma medida direta da distância euclidiana, 
    que é a mais comum em um espaço cartesiano. No entanto, quando se trata de calcular a distância de um ponto a uma linha,
    a fórmula da linha é mais apropriada, pois leva em consideração a inclinação da linha e a posição do ponto em relação a ela.
    
    Falando de forma informal: 
    Eles medem coisas diferentes. A distância ponto-reta nunca vai ser a mesma que a distância entre dois pontos específicos, porque:
    Uma compara ponto com ponto, A outra compara ponto com linha infinita.

    """
    return abs(a * novo_ponto[0] + b * novo_ponto[1] + c) / ((a**2 + b**2) ** 0.5)

print(f"Distancia usando {distancia_ponto_reta((2, 2), *coeficientes_reta(A, B))}")

Distancia usando 0.0


In [110]:
def projecao_no_segmento(p: tuple[int, int], a: int, b: int, c: int, p1: tuple[int, int], p2: tuple[int, int]) -> tuple[float, float] | None:
    """
    Retorna a projeção ortogonal do ponto p sobre a reta definida por (a, b, c),
    mas somente se a projeção cair dentro do segmento [p1, p2].
    Caso contrário, retorna None.
    
    link aula: https://www.youtube.com/watch?v=fem35PVZjeA
    link solução:  https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line
    

    oque é o denominador?
    O denominador é a soma dos quadrados dos coeficientes a e b da reta.
    Ele é usado para calcular as coordenadas da projeção ortogonal do ponto p na reta.
    Se o denominador for zero, significa que a reta é vertical (ou seja, a = 0 e b = 0), e a projeção não pode ser calculada.
    
    Falando de forma leiga:
    Estamos trabalhando com sombras, onde a reta é a superfície e o ponto p é a fonte de luz. A projeção ortogonal é a "sombra" do ponto p na reta.
    Isso significa que estamos tentando encontrar o ponto na reta que está diretamente abaixo do ponto p, como se estivéssemos projetando a luz do ponto p na reta.
    """
    x, y = p
    denominador = a*a + b*b
    if denominador == 0:
        raise ValueError("Coeficientes inválidos para reta.")

    px = (b * (b*x - a*y) - a*c) / denominador
    py = (a * (-b*x + a*y) - b*c) / denominador

    # Verifica se a projeção está dentro do segmento
    min_x, max_x = min(p1[0], p2[0]), max(p1[0], p2[0])
    min_y, max_y = min(p1[1], p2[1]), max(p1[1], p2[1])
    if min_x <= px <= max_x and min_y <= py <= max_y:
        return px, py
    return None


In [111]:
def reta_entre_dois_pontos(centroids: list[tuple], novo_ponto: tuple[int, int], novo_destino: tuple[int, int]):
  """
  Verifica se é possivel ter uma reta entre dois pontos sem que um centróide a intercepte.

  """
  a, b, c = coeficientes_reta(novo_ponto, novo_destino)
  for centroid in centroids:
    distancia = distancia_ponto_reta(centroid, a, b, c)
    if distancia < raio_size:
        if (centroid[0] - novo_ponto[0]) * (novo_destino[1] - novo_ponto[1]) == (centroid[1] - novo_ponto[1]) * (novo_destino[0] - novo_ponto[0]):
            print("Centróide está na reta")
            return False

        if projecao_no_segmento(centroid, a, b, c, novo_ponto, novo_destino):
            print("Centróide está na projeção da reta")
            return False

  return True

In [112]:
retas = []

todos_pontos = [ponto_inicial_final[0]]
for x in range(len(posicoes_centroides)):
    for y in range(len(posicoes_centroides[x])):
        todos_pontos.append(posicoes_centroides[x][y])
todos_pontos.append(ponto_inicial_final[1])

for i in range(len(todos_pontos)):
    for j in range(i+1, len(todos_pontos)):
        if reta_entre_dois_pontos(centroids, todos_pontos[i], todos_pontos[j]):
            retas.append((todos_pontos[i], todos_pontos[j]))

retas_desenhadas = []
for reta in retas:
    exibir_figura(fig)
    retas_desenhadas.append(ax.plot(*zip(*reta), color='black', linewidth=0.5))

print(f"Total de pontos: {len(todos_pontos)}")
print(f"Total de retas criadas: {len(retas)}")


Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na projeção da reta
Centróide está na pr